In [1]:
import pandas as pd
import warnings
import math
from matplotlib import pyplot as plt
from datetime import datetime
import sys
import numpy as np

In [2]:
print("Hora:", datetime.now().strftime("%H:%M:%S"))

Hora: 09:10:35


In [3]:
states = ['AC','AL','AM','AP','BA','CE','DF','ES','GO','MA','MG','MS','MT','PA','PB','PE','PI','PR','RJ','RN','RO','RR','RS','SC','SE','SP','TO']
years = list(range(2014,2025))

# general path
general_path = 'C:/Users/Windows/OneDrive/Documentos/mestrado/DISSERTAÇÃO/scripts'
mit_int=360
metodo = "RUSLE2"   # Escolha entre "USLE", "RUSLE" ou "RUSLE2"


pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)



#LISTA DE HQ
df_tel_hq=pd.read_excel(general_path+'/Estacoes_telemetria_HQ.xlsx')
df_cem_hq=pd.read_csv(general_path+'/Estacoes_Alta_Qualidade_da_Analise_Visual.csv',sep=';')

In [4]:
def calcular_KE(I, metodo):
    if metodo.upper() == "USLE":
        if I <= 76:
            return 0.119 + 0.0873 * math.log10(I)
        else:
            return 0.283
    elif metodo.upper() == "RUSLE":
            return 0.29 * (1 - 0.72 * math.exp(-0.05 * I))
    elif metodo.upper() == "RUSLE2":
        return 0.29 * (1 - 0.72 * math.exp(-0.082 * I))

In [ ]:
for state in states:
    for year in years:
        start_time = datetime.now()
        mit=pd.Timedelta(mit_int, unit ='m')
        name_file= state+ '_'+ str(year)+'.h5'
        name_col= state+ '_ '+ str(year)
        print(f"Começando : {name_col} {start_time}") 
        df_rainfall = pd.read_hdf(general_path +'/1 - Organized data/'+ name_file,'table_data')

        monthly_results = {m: {"gauge_code": [], "EI30": [], "E_D": [], "rain": []} for m in range(1, 13)}
        annual_results = {"gauge_code": [], "EI30": [], "E_D": [], "rain": []}
        for code in df_cem_hq[name_col].dropna():
            start_time_code = datetime.now()  
            name=code +'_' + str(mit_int)
            df_rainfall_events = pd.read_hdf(general_path + '/3 - Rainfall events/'+ name_file, name).drop_duplicates()
            df_partial_result = pd.DataFrame(columns=['rain_mm','duration', 'I30', 'RF_KE', 'EI30'])
            for evento in range(0,len(df_rainfall_events)):
                beg=df_rainfall_events.iloc[evento,1]
                month = beg.month
                end=df_rainfall_events.iloc[evento,1]+pd.Timedelta(hours=df_rainfall_events.iloc[evento,3])
                df_gauge_rainfall=df_rainfall[df_rainfall['gauge_code']==code]
                df_event = df_gauge_rainfall[(df_gauge_rainfall['datetime'] >= beg) & (df_gauge_rainfall['datetime'] <= end)].reset_index(drop=True)

                if df_event['rain_mm'].sum()>12.7:  #especificação RUSLE 2
                    for i in df_event.index:
                        RF=df_event['rain_mm'][i] #mm
                        RF_intensity=(RF/10)*60 #mm/h  ###tdel
                        #kinectic energy
                        if RF==0:KE=0 
                        else:
                            KE=calcular_KE(RF_intensity, metodo)
                            RF_KE=KE*RF#MJ/ha
                            if (KE > 0.283) :KE= 0.283 #VALOR MÁXIMO
    
                            tbeg=df_event.iloc[i,3] #df_temp é para os 30 min de análise para o i30
                            tend=df_event.iloc[i,3]+pd.Timedelta(hours=0.5)
                            df_temp=df_event[(df_event['datetime'] >= tbeg) & (df_event['datetime'] < tend)]#### < ou <=
                            I30=df_temp['rain_mm'].sum()/0.5 #mm/h #intervalo de 0,5 hrs
    
                            df_event.at[i,'Rf_intensity']=RF_intensity
                            df_event.at[i,'RF_KE']=RF_KE
                            df_event.at[i,'I30']=I30

                    #calc ei30       
                    EI30=df_event['RF_KE'].sum()*df_event['I30'].max()        
                    #set partial results
                    df_partial_result.at[evento,'rain_mm']=df_event['rain_mm'].sum() 
                    df_partial_result.at[evento,'duration']=df_rainfall_events.iloc[evento,3]        
                    df_partial_result.at[evento,'I30']=df_event['I30'].max() 
                    df_partial_result.at[evento,'RF_KE']=df_event['RF_KE'].sum()
                    df_partial_result.at[evento,'EI30']=EI30
                    df_partial_result.at[evento,'month']=month
                
                else:

                    df_partial_result.at[evento,'rain_mm']=df_rainfall_events.iloc[evento,2]  
                    df_partial_result.at[evento,'duration']=df_rainfall_events.iloc[evento,3]        
                    df_partial_result.at[evento,'I30']=0
                    df_partial_result.at[evento,'RF_KE']=0
                    df_partial_result.at[evento,'EI30']=0
                    df_partial_result.at[evento,'month']=month
                #set final results        
            with pd.HDFStore(general_path + f'/1 - Results_Cemaden/Results_{name_col}.h5', 'a') as store:
                for month in range(1, 13):

                    df_month = df_partial_result[df_partial_result['month']==month]
                    EI30_M=df_month['EI30'].sum()
                    rain=df_month['rain_mm'].sum()
                    E_D = EI30_M / rain if rain != 0 else float('nan')
                    monthly_results[month]["gauge_code"].append(code)
                    monthly_results[month]["EI30"].append(EI30_M)
                    monthly_results[month]["E_D"].append(E_D)
                    monthly_results[month]["rain"].append(rain)                
                    
                    store.put(f'month_{month:02d}',pd.DataFrame(monthly_results[month]))
            
                #set anual results
                
                EI30_T=df_partial_result['EI30'].sum()
                E_D=EI30_T/df_partial_result['rain_mm'].sum()
                rain=df_partial_result['rain_mm'].sum()
                annual_results["gauge_code"].append(code)
                annual_results["EI30"].append(EI30_T)
                annual_results["E_D"].append(E_D)
                annual_results["rain"].append(rain)
                store.put('annual', pd.DataFrame(annual_results))


            print(f"  -code: {code} , tempo: {(datetime.now()-start_time_code).total_seconds():.2f}s , {len(annual_results['E_D'])}/{len(df_cem_hq[name_col].dropna())}")

        minutes, seconds = divmod((datetime.now()-start_time).total_seconds(), 60)#duração total
        print(f"Finalizado: {name_col}, Duração_total: {int(minutes)}m {seconds:.2f}s")         

Começando : AC_ 2014 2025-03-27 10:18:31.886795
Finalizado: AC_ 2014, Duração_total: 0m 0.08s
Começando : AC_ 2015 2025-03-27 10:18:31.967274
Finalizado: AC_ 2015, Duração_total: 0m 0.02s
Começando : AC_ 2016 2025-03-27 10:18:31.983088
Finalizado: AC_ 2016, Duração_total: 0m 0.03s
Começando : AC_ 2017 2025-03-27 10:18:32.015342
Finalizado: AC_ 2017, Duração_total: 0m 0.03s
Começando : AC_ 2018 2025-03-27 10:18:32.042365
Finalizado: AC_ 2018, Duração_total: 0m 0.03s
Começando : AC_ 2019 2025-03-27 10:18:32.072174
  -code: 120010401A , tempo: 2.09s , 1/2
  -code: 120070801A , tempo: 2.08s , 2/2
Finalizado: AC_ 2019, Duração_total: 0m 4.22s
Começando : AC_ 2020 2025-03-27 10:18:36.293651
  -code: 120070801A , tempo: 1.76s , 1/1
Finalizado: AC_ 2020, Duração_total: 0m 1.82s
Começando : AC_ 2021 2025-03-27 10:18:38.115177
  -code: 120010401A , tempo: 1.98s , 1/2
  -code: 120070801A , tempo: 2.03s , 2/2
Finalizado: AC_ 2021, Duração_total: 0m 4.09s
Começando : AC_ 2022 2025-03-27 10:18:42.20

In [48]:
month

10

In [49]:
df_partial_result

,rain_mm,duration,I30,RF_KE,EI30,month
0,0.4,0.5,0,0,0,1.0
1,7.4,0.666667,0,0,0,1.0
2,1.4,0.166667,0,0,0,2.0
3,0.2,0.166667,0,0,0,2.0
4,9.8,2.833333,0,0,0,2.0
5,6.0,2.5,0,0,0,2.0
6,14.0,13.0,9.2,2.276426,20.943115,2.0
7,3.2,2.0,0,0,0,2.0
8,0.2,0.166667,0,0,0,2.0
9,3.0,7.666667,0,0,0,2.0


In [ ]:
with pd.HDFStore(general_path + f'/1 - Results_Cemaden/Results_{'AC_ 2019'}.h5') as hdf:
    keys = hdf.keys()  # Obtém todas as chaves do arquivo HDF
    for key in keys:  # Itera por cada chave
        df = pd.read_hdf(hdf, key=key)  # Lê os dados de cada chave
        print(f"Chave: {key}, Número de linhas: {len(df)}")


import h5py

# Abrir o arquivo HDF5 no modo leitura/escrita
with h5py.File(general_path + f'/1 - Results_Cemaden/Results_{'AC_ 2019'}.h5', "r+") as f:
    chave = "month_13"  # Nome da chave a ser removida
    if chave in f:
        del f[chave]  # Remove a chave do HDF5
        print(f"Chave '{chave}' removida com sucesso!")
    else:
        print(f"Chave '{chave}' não encontrada no arquivo.")


In [45]:
with pd.HDFStore(general_path + f'/1 - Results_Cemaden/Results_{'AL_ 2017'}.h5') as hdf:
    keys = hdf.keys()  # Obtém todas as chaves do arquivo HDF
    for key in keys:  # Itera por cada chave
        df = pd.read_hdf(hdf, key=key)  # Lê os dados de cada chave
        print(f"Chave: {key}, Número de linhas: {len(df)}")


Chave: /annual, Número de linhas: 21
Chave: /month_01, Número de linhas: 21
Chave: /month_02, Número de linhas: 21
Chave: /month_03, Número de linhas: 21
Chave: /month_04, Número de linhas: 21
Chave: /month_05, Número de linhas: 21
Chave: /month_06, Número de linhas: 21
Chave: /month_07, Número de linhas: 21
Chave: /month_08, Número de linhas: 21
Chave: /month_09, Número de linhas: 21
Chave: /month_10, Número de linhas: 21
Chave: /month_11, Número de linhas: 21
Chave: /month_12, Número de linhas: 21


In [7]:
# general path
general_path = 'C:/Users/Windows/OneDrive/Documentos/mestrado/DISSERTAÇÃO/scripts'
general_path_tel='C:/Users/Windows/OneDrive/Documentos/mestrado/Telemetria-ANA/Scripts e dados'

states = ['MT']

years = [2020]
mit_int=360

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#LISTA DE HQ
df_tel_hq=pd.read_excel(general_path+'/Estacoes_telemetria_HQ.xlsx')

In [8]:
for state in states:
    for year in years:
        start_time = datetime.now()
        name_col=state+ '_ '+ str(year)
        print(f"Começando : {name_col} - {start_time}") 
        mit=pd.Timedelta(mit_int, unit ='m')
        name_file= state+ '_'+ str(year)+'.h5'
        
        df_info = pd.read_hdf(general_path_tel +'/1 - Organized data/'+ name_file,'table_info')
        df_year_hq=df_tel_hq[year].dropna().astype(int)
        common_values = set(df_info['gauge_code'].astype(int)).intersection(set(df_year_hq))
        hq_name_file=list(map(str, common_values))
        df_rainfall = pd.read_hdf(general_path_tel +'/1 - Organized data/'+ name_file,'table_data').dropna(subset=['rain_mm']).reset_index(drop=True)
        data = {"gauge_code": [],"EI30_T": [],"E_D": []}
        
        
        for code in hq_name_file:
            start_time_code = datetime.now()  
            name=code +'_' + str(mit_int)
            df_rainfall_events = pd.read_hdf(general_path_tel + '/3 - Rainfall events/'+ name_file, name)
            df_partial_result = pd.DataFrame(columns=['rain_mm','duration', 'I30', 'RF_KE', 'EI30'])
            for evento in range(0,len(df_rainfall_events)):
                beg=df_rainfall_events.iloc[evento,1]
                end=df_rainfall_events.iloc[evento,1]+pd.Timedelta(hours=df_rainfall_events.iloc[evento,3])
                df_gauge_rainfall=df_rainfall[df_rainfall['gauge_code']==code]
                df_event = df_gauge_rainfall[(df_gauge_rainfall['datetime'] >= beg) & (df_gauge_rainfall['datetime'] <= end)].reset_index(drop=True)
                for i in df_event.index:
                    RF=df_event['rain_mm'][i] #mm
                    time_delta=int((df_event.iloc[i,3]-df_event.iloc[i-1,3]).total_seconds() // 60)
                    if i==0:RF_intensity=(RF/30)*60 #mm/h  ###tdel
                    elif time_delta==0:RF_intensity=(RF/15)*60 #mm/h  ###tdel
                    else:RF_intensity=(RF/time_delta)*60 #mm/h  ###tdel
                    #kinectic energy
                    if RF<=0:KE=0 
                    else:
                        KE=0.119+0.0873*math.log(RF_intensity,10)  #KE= 1099*(1-0.72*math.exp(-2.088*RF_intensity)) ##MJ/ha.mm
                        RF_KE=KE*RF#MJ/ha
                        if (KE > 0.283) :KE= 0.283

                        tbeg=df_event.iloc[i,3] #df_temp é para os 30 min de análise para o i30
                        tend=df_event.iloc[i,3]+pd.Timedelta(hours=0.5)
                        df_temp=df_event[(df_event['datetime'] >= tbeg) & (df_event['datetime'] < tend)]#### < ou <=
                        I30=df_temp['rain_mm'].sum()/0.5 #mm/h #intervalo de 0,5 hrs

                        df_event.at[i,'Rf_intensity']=RF_intensity
                        df_event.at[i,'RF_KE']=RF_KE
                        df_event.at[i,'I30']=I30

                #calc ei30       
                EI30=df_event['RF_KE'].sum()*df_event['I30'].max()        
                #set partial results
                df_partial_result.at[evento,'rain_mm']=df_event['rain_mm'].sum() 
                df_partial_result.at[evento,'duration']=df_rainfall_events.iloc[evento,3]        
                df_partial_result.at[evento,'I30']=df_event['I30'].max() 
                df_partial_result.at[evento,'RF_KE']=df_event['RF_KE'].sum()
                df_partial_result.at[evento,'EI30']=EI30 

            #set final results
            EI30_T=df_partial_result['EI30'].sum()
            E_D=EI30_T/df_partial_result['rain_mm'].sum()
            data["gauge_code"].append(code)
            data["EI30_T"].append(EI30_T)
            data["E_D"].append(E_D)


            end_time = datetime.now()  # Fim do tempo de iteração
            print(f"  -code: {code} , tempo: {(end_time-start_time_code).total_seconds():.2f}s , {len(data['E_D'])}/{len(hq_name_file)}, ei3o={EI30_T}")
        df_name_col = pd.DataFrame(data)
        df_name_col.to_excel(general_path_tel+f'/1 - Results_Telemetria/Results_{name_col}.xlsx', index=False)
        minutes, seconds = divmod((end_time-start_time).total_seconds(), 60)
        print(f"Finalizado: {name_col}, Duração_total: {int(minutes)}m {seconds:.2f}s ")

Começando : MT_ 2020 - 2025-01-13 18:15:04.301570
  -code: 66384000 , tempo: 29.58s , 1/54, ei3o=10840.539000797375
  -code: 17094400 , tempo: 26.70s , 2/54, ei3o=12111.344637889193
  -code: 18409350 , tempo: 28.05s , 3/54, ei3o=12722.19107653221
  -code: 15123080 , tempo: 34.20s , 4/54, ei3o=7817.684057394804
  -code: 26033800 , tempo: 24.96s , 5/54, ei3o=6917.620314759616
  -code: 17091600 , tempo: 19.94s , 6/54, ei3o=13627.39043010723
  -code: 17091090 , tempo: 28.11s , 7/54, ei3o=3658.445157671127
  -code: 17093010 , tempo: 30.00s , 8/54, ei3o=6935.491623994871
  -code: 17094520 , tempo: 27.24s , 9/54, ei3o=16059.057233763062
  -code: 24179090 , tempo: 25.97s , 10/54, ei3o=7855.786620903551
  -code: 17094550 , tempo: 31.82s , 11/54, ei3o=11620.891399260954
  -code: 66025500 , tempo: 23.18s , 12/54, ei3o=10888.81378199183
  -code: 18420000 , tempo: 21.15s , 13/54, ei3o=11707.86953706867
  -code: 26100000 , tempo: 24.01s , 14/54, ei3o=12896.273798753216
  -code: 66420000 , tempo: 19.

  -code: 75343000 , tempo: 19.38s , 8/39, ei3o=9210.624656613038
  -code: 87380000 , tempo: 32.05s , 9/39, ei3o=6697.616098659433
  -code: 75188000 , tempo: 35.95s , 10/39, ei3o=10810.287328904495
  -code: 76150050 , tempo: 22.73s , 11/39, ei3o=7651.168493222472
  -code: 88260000 , tempo: 31.28s , 12/39, ei3o=7609.085557328341
  -code: 74761000 , tempo: 40.49s , 13/39, ei3o=11374.36835768795
  -code: 88060200 , tempo: 28.44s , 14/39, ei3o=4093.996918092736
  -code: 86410800 , tempo: 53.55s , 15/39, ei3o=6385.702073449372
  -code: 75342000 , tempo: 19.81s , 16/39, ei3o=11427.850298431667
  -code: 73420080 , tempo: 22.03s , 17/39, ei3o=9190.04094749943
  -code: 75550000 , tempo: 28.03s , 18/39, ei3o=10154.935965084267
  -code: 86110000 , tempo: 25.62s , 19/39, ei3o=6829.005697609062
  -code: 86280500 , tempo: 28.47s , 20/39, ei3o=7307.106389002127
  -code: 73390000 , tempo: 38.89s , 21/39, ei3o=7216.154119967222
  -code: 86410050 , tempo: 30.08s , 22/39, ei3o=8244.573321071903
  -code: 8

  -code: 86100600 , tempo: 38.85s , 34/35, ei3o=11316.758174190789
  -code: 86125050 , tempo: 38.07s , 35/35, ei3o=24201.506774288082
Finalizado: RS_ 2023, Duração_total: 22m 31.13s 
Começando : SC_ 2014 - 2025-01-13 03:34:54.027437
  -code: 83880000 , tempo: 25.68s , 1/9, ei3o=9882.243789775775
  -code: 84538500 , tempo: 15.08s , 2/9, ei3o=13585.60153043693
  -code: 65293000 , tempo: 28.36s , 3/9, ei3o=9022.411706622619
  -code: 83629000 , tempo: 23.89s , 4/9, ei3o=9226.373607954109
  -code: 84095500 , tempo: 22.87s , 5/9, ei3o=10293.907644903024
  -code: 84580500 , tempo: 21.25s , 6/9, ei3o=6637.296450984035
  -code: 65259000 , tempo: 21.38s , 7/9, ei3o=8679.563339754879
  -code: 65275000 , tempo: 18.98s , 8/9, ei3o=9761.823959143723
  -code: 83892990 , tempo: 23.05s , 9/9, ei3o=11125.36497047818
Finalizado: SC_ 2014, Duração_total: 3m 27.89s 
Começando : SC_ 2015 - 2025-01-13 03:38:21.963304
  -code: 74324000 , tempo: 28.18s , 1/5, ei3o=12027.158628673798
  -code: 73620000 , tempo: 

  -code: 84538500 , tempo: 20.90s , 4/32, ei3o=4245.994826823322
  -code: 73674500 , tempo: 33.16s , 5/32, ei3o=9843.417258332818
  -code: 84853000 , tempo: 33.24s , 6/32, ei3o=5294.574978162448
  -code: 72810000 , tempo: 28.75s , 7/32, ei3o=8924.673085483344
  -code: 84580500 , tempo: 25.76s , 8/32, ei3o=5729.541127822922
  -code: 84079000 , tempo: 29.55s , 9/32, ei3o=10089.872051652685
  -code: 84015900 , tempo: 29.26s , 10/32, ei3o=5127.40352305633
  -code: 73892000 , tempo: 28.86s , 11/32, ei3o=9547.118525725766
  -code: 71540000 , tempo: 32.70s , 12/32, ei3o=5475.933041133671
  -code: 84030000 , tempo: 20.56s , 13/32, ei3o=4559.327730550927
  -code: 72860080 , tempo: 21.67s , 14/32, ei3o=7669.940343497607
  -code: 73582000 , tempo: 26.73s , 15/32, ei3o=15611.45488851415
  -code: 83059000 , tempo: 24.05s , 16/32, ei3o=10020.49153147309
  -code: 84531000 , tempo: 21.41s , 17/32, ei3o=6583.719697503275
  -code: 73960000 , tempo: 25.73s , 18/32, ei3o=12656.217687001632
  -code: 845360

  -code: 73693500 , tempo: 43.46s , 27/46, ei3o=13783.600058537702
  -code: 73960000 , tempo: 34.08s , 28/46, ei3o=10549.370690280895
  -code: 84536000 , tempo: 21.87s , 29/46, ei3o=10055.39879430916
  -code: 70600000 , tempo: 35.95s , 30/46, ei3o=12744.46908424854
  -code: 65293000 , tempo: 31.80s , 31/46, ei3o=5648.71683523661
  -code: 73331800 , tempo: 29.77s , 32/46, ei3o=5553.981630731354
  -code: 83349600 , tempo: 31.68s , 33/46, ei3o=9004.151509040425
  -code: 83900000 , tempo: 29.58s , 34/46, ei3o=10201.183514152499
  -code: 71388000 , tempo: 36.48s , 35/46, ei3o=7332.292845829894
  -code: 71900000 , tempo: 29.48s , 36/46, ei3o=7757.792853634563
  -code: 72849000 , tempo: 33.55s , 37/46, ei3o=11550.90521909616
  -code: 73693800 , tempo: 38.86s , 38/46, ei3o=9076.307571065478
  -code: 73249000 , tempo: 51.08s , 39/46, ei3o=11620.44296565999
  -code: 84075000 , tempo: 30.92s , 40/46, ei3o=8856.115457776521
  -code: 83441000 , tempo: 24.27s , 41/46, ei3o=6177.649941214009
  -code:

  -code: 62473200 , tempo: 25.00s , 25/25, ei3o=6120.377025890221
Finalizado: SP_ 2017, Duração_total: 10m 3.66s 
Começando : SP_ 2018 - 2025-01-13 06:16:01.375792
  -code: 62848000 , tempo: 15.27s , 1/20, ei3o=8947.945742626425
  -code: 64080000 , tempo: 19.72s , 2/20, ei3o=7708.680201002242
  -code: 81168000 , tempo: 29.19s , 3/20, ei3o=7214.569433734796
  -code: 64320010 , tempo: 14.98s , 4/20, ei3o=7305.816931664827
  -code: 61895050 , tempo: 16.32s , 5/20, ei3o=5190.8293600772795
  -code: 61775000 , tempo: 18.44s , 6/20, ei3o=4105.874813854197
  -code: 61783450 , tempo: 16.99s , 7/20, ei3o=12214.571149799693
  -code: 64250010 , tempo: 17.80s , 8/20, ei3o=5218.03549237307
  -code: 81337000 , tempo: 27.11s , 9/20, ei3o=7886.52884424337
  -code: 63170100 , tempo: 16.84s , 10/20, ei3o=3104.7627818806245
  -code: 58212150 , tempo: 30.54s , 11/20, ei3o=5614.036372726824
  -code: 61783100 , tempo: 26.29s , 12/20, ei3o=27616.08660447711
  -code: 62824000 , tempo: 13.52s , 13/20, ei3o=3829

  -code: 62824000 , tempo: 33.58s , 18/31, ei3o=6318.453565369782
  -code: 64232000 , tempo: 52.00s , 19/31, ei3o=6785.576546480001
  -code: 61880000 , tempo: 37.96s , 20/31, ei3o=8393.555804621334
  -code: 62344000 , tempo: 58.32s , 21/31, ei3o=7146.897036636748
  -code: 62557000 , tempo: 42.99s , 22/31, ei3o=3995.031868664784
  -code: 61997000 , tempo: 34.98s , 23/31, ei3o=12926.884669959665
  -code: 62344010 , tempo: 57.88s , 24/31, ei3o=7146.897036636748
  -code: 64098000 , tempo: 65.64s , 25/31, ei3o=9417.294693284792
  -code: 61874000 , tempo: 40.63s , 26/31, ei3o=8481.556579674087
  -code: 62400600 , tempo: 42.48s , 27/31, ei3o=7609.979826529602
  -code: 64278500 , tempo: 44.67s , 28/31, ei3o=9408.1423403336
  -code: 62758000 , tempo: 39.22s , 29/31, ei3o=7869.7228160881905
  -code: 58235000 , tempo: 48.45s , 30/31, ei3o=5392.140854293327
  -code: 62360060 , tempo: 55.08s , 31/31, ei3o=5169.690965184644
Finalizado: SP_ 2022, Duração_total: 23m 58.05s 
Começando : SP_ 2023 - 2025